# Save, Load and Predict Using the Sonar Model

In [3]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

seed = 7
numpy.random.seed(seed)

dataframe = pandas.read_csv("03-sonar.csv", delimiter=',', header=None)
dataset = dataframe.values

X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)

def create_baseline(optimizer='adam', init='normal'):
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, encoded_y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Using TensorFlow backend.


Results: 82.69% (8.09%)


In [4]:
estimator.fit(X,encoded_y)

#save model as Json
model_json = estimator.model.to_json()
with open("model_sonar.json", "w") as json_file:
    json_file.write(model_json)

#save the weights as hdf5
estimator.model.save_weights("model_sonar.h5")

print("Saved the model to the computer")

Saved the model to the computer


In [5]:
from keras.models import model_from_json

#load the json from the file and recreate the model
json_file = open('model_sonar.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load the weights from hdf5 file
loaded_model.load_weights("model_sonar.h5")
print("Loaded model from the computer")

Loaded model from the computer


In [7]:
from numpy import array

# load weights into new model
loaded_model.load_weights("model_sonar.h5")
print("Loaded model from disk")

# predict the output from new instance where we do not know the answer
Xnew = array([[0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032]])

# make a prediction
ynew = loaded_model.predict_classes(Xnew)
# show the inputs and predicted outputs
print("X=%s, Predicted=%s" % (Xnew[0], encoder.inverse_transform(ynew[0])))


Loaded model from disk
X=[0.02   0.0371 0.0428 0.0207 0.0954 0.0986 0.1539 0.1601 0.3109 0.2111
 0.1609 0.1582 0.2238 0.0645 0.066  0.2273 0.31   0.2999 0.5078 0.4797
 0.5783 0.5071 0.4328 0.555  0.6711 0.6415 0.7104 0.808  0.6791 0.3857
 0.1307 0.2604 0.5121 0.7547 0.8537 0.8507 0.6692 0.6097 0.4943 0.2744
 0.051  0.2834 0.2825 0.4256 0.2641 0.1386 0.1051 0.1343 0.0383 0.0324
 0.0232 0.0027 0.0065 0.0159 0.0072 0.0167 0.018  0.0084 0.009  0.0032], Predicted=['R']
